In [3]:
%cd /scratch/bruingjde/SNAM2021-code/

import collections, os, typing

import joblib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import sklearn.linear_model
import sklearn.model_selection
import sklearn.metrics
import sklearn.pipeline
import sklearn.preprocessing
from tqdm.auto import tqdm

import tlp



feature = 'jc'
time_strategies = tlp.TIME_STRATEGIES
aggregation_strategies = tlp.AGGREGATION_STRATEGIES

hypergraph = {1, 2, 3, 5, 6, 7, 12, 13, 14, 19, 22, 23, 25, 26, 28, 29, 30}
simplegraph = {4, 8, 9, 10, 11, 15, 16, 17, 18, 20, 21, 24, 27}

/scratch/bruingjde/SNAM2021-code


In [4]:
args = [
  dict(index=index, time_str=time_str, time_func=time_func, agg_str=agg_str, agg_func=agg_func)
  for index in hypergraph
  for time_str, time_func in time_strategies.items()
  for agg_str, agg_func in aggregation_strategies.items()
] + [
  dict(index=index, time_str=time_str, time_func=time_func)
  for index in simplegraph
  for time_str, time_func in time_strategies.items()
]

In [5]:
args[35]

{'index': 2,
 'time_str': 'lin',
 'time_func': <function tlp.getFeatures.strategies.lin(x: pandas.core.series.Series, lower_bound=0.2)>,
 'agg_str': 'm3',
 'agg_func': <function scipy.stats.stats.kurtosis(a, axis=0, fisher=True, bias=True, nan_policy='propagate')>}

In [7]:
index = args[35]['index']
edgelist_mature = pd.read_pickle(os.path.join('data', f'{index:02}', 'edgelist_mature.pkl'))
instances = np.load(os.path.join(os.path.join('data', f'{index:02}', 'instances_sampled.npy')))

In [8]:
edgelist_mature['datetime_transformed'] = args[35]['time_func'](edgelist_mature['datetime'])

In [9]:
G = nx.from_pandas_edgelist(edgelist_mature, create_using=nx.MultiGraph, edge_attr=True)

In [10]:
agg_func = args[35]['agg_func']

In [ ]:
scores = [
  (
      sum([agg_func([edge_attributes['datetime_transformed'] for edge_attributes in G.get_edge_data(u, a).values()]) for a in G[u]]) +
      sum([agg_func([edge_attributes['datetime_transformed'] for edge_attributes in G.get_edge_data(v, b).values()]) for b in G[v]])
    )
  for u, v in tqdm(instances)
]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [19]:
np.min(scores)

-1412.5746890280057